In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:

import sys
from itertools import groupby
import numpy as np
import cv2
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
from glob import glob
from multiprocessing import Pool
import matplotlib.pyplot as plt
# import cupy as cp
import ast
from pathlib import Path
import pickle5 as pickle
import torch

import shutil
import sys
sys.path.append("../src/")
sys.path.append("../yolov5/")

import util

from joblib import Parallel, delayed

from IPython.display import display, HTML
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from ensemble_boxes import weighted_boxes_fusion

from IPython.display import Image
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
CONF = 0
IOU = 0.4
AUGMENT = True # True
SIZE = int(1280 * 1.3)  # 1280 * 1.3 
#USE_CLACHE = True
USE_SAHI = False

In [ ]:
INPUT_DIR = Path("../../data/tensorflow-great-barrier-reef/")
df = pd.read_csv(INPUT_DIR / "train.csv")
#folds = pd.read_csv("../input/train_folds_balanced.csv") #_CPF
#df = pd.merge(df, folds[["video_id","video_frame","fold"]], on=["video_id",'video_frame'])
data_param = {'root_dir':INPUT_DIR,'label_dir':INPUT_DIR / "labels"}
df = df.progress_apply(lambda x: util.get_path(x, data_param, infer=True), axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df["real_bbox"] = df["annotations"].apply(lambda annots: [list(annot.values()) for annot in annots])
df.shape

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

# Folds

In [ ]:
#save it
# A for Anjum, C for CPF, V for video
df_l_A_cv = util.load_pickle("pred_all_large_wpre.pkl")
df_l_V_cv = util.load_pickle("pred_all_large_wpre_video.pkl")
df_l_V_cv_FP = util.load_pickle("pred_all_large_wpre_video_FP.pkl")
df_s_A_cv = util.load_pickle("pred_all.pkl")
df_s_C_cv = util.load_pickle("pred_all_cpf.pkl")

In [ ]:
folds_A = pd.read_csv("../input/train_folds_balanced.csv") 
folds_C = pd.read_csv("../input/train_folds_balanced_CPF.csv")
df['fold_V'] = df["video_id"]
df["fold_A"] = folds_A["fold"]
df["fold_C"] = folds_C["fold"]

In [ ]:
#df_plain["l_A"] = np.nan
#df_plain["l_V"] = np.nan
#df_plain["s_A"] = np.nan
#df_plain["s_C"] = np.nan

# th_d = {
#     "l_A" : 0.1,
#     "l_V" : 0.1,
#     "s_A" : 0.2,
#     "s_C" : 0.2,
# }
df_d = {
    "l_A" : df_l_A_cv,
    "l_V" : df_l_V_cv,
    "l_V_FP" : df_l_V_cv_FP,
    "s_A" : df_s_A_cv,
    "s_C" : df_s_C_cv,    
}
for key in tqdm(df_d.keys()):
    print(key, df.shape)
    df = pd.merge(df, df_d[key][["video_id","video_frame", "pred"]].rename(columns={"pred":key}), on=["video_id","video_frame"])
    df.dropna(subset=key,inplace=True)
    #df[key] = df[key].apply(lambda x: [p for p in x if p[0] >= th_d[key]])

In [ ]:
gt_bboxs_list = []
for idx in range(len(df)):
    row = df.iloc[idx]
    gt_bboxs = row["real_bbox"]
    gt_bboxs_list.append(np.array(gt_bboxs))
len(gt_bboxs_list)

## Get threshold

In [ ]:
conf_ths = np.arange(5, 35, 5) / 100
perf_dict = {}
for key in tqdm(df_d.keys()):
    if key not in perf_dict:
        perf_dict[key] = {}
    for conf_th in conf_ths:
        prd_bboxs_list = []
        for index in df.index:
            prd_bboxs = df.loc[index, key]
            prd_bboxs_list.append(np.array([p for p in prd_bboxs if p[0] >= conf_th]))
        f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
        print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(key, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
        del f2_dict["tps"]
        del f2_dict["fps"]
        del f2_dict["fns"]
        perf_dict[key][conf_th] = f2_dict

In [ ]:
th_d = {
    "l_A" : 0.1,
    "l_V" : 0.1,
    "l_V_FP" : 0.15,
    "s_A" : 0.2,
    "s_C" : 0.25,
}

In [ ]:
np.sort([3,2])

In [ ]:
perf_dict_best = {}
for key in tqdm(df_d.keys()):
    conf_th = th_d[key]
    prd_bboxs_list = []
    for index in df.index:
        prd_bboxs = df.loc[index, key]
        prd_bboxs_list.append(np.array([p for p in prd_bboxs if p[0] >= conf_th]))
    df[key] = [p.tolist() for p in prd_bboxs_list]
    f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
    print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(key, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    df[f'{key}_tps'] = f2_dict["tps"]
    df[f'{key}_fps'] = f2_dict["fps"]
    df[f'{key}_fns'] = f2_dict["fns"]
    del f2_dict["tps"]
    del f2_dict["fps"]
    del f2_dict["fns"]
    perf_dict_best[key] = f2_dict
    
    fold_key = key.split("_")[1]
    fold_col = f"fold_{fold_key}"
    for fold in np.sort(df[fold_col].unique()):
        df_fold = df.query(f"{fold_col} == {fold}")
        
        
        gt_bboxs_fold_list = []
        for idx in df_fold.index:
            row = df_fold.loc[idx]
            gt_bboxs = row["real_bbox"]
            gt_bboxs_fold_list.append(np.array(gt_bboxs))
            
        prd_bboxs_list = []
        for index in df_fold.index:
            prd_bboxs = df_fold.loc[index, key]
            prd_bboxs_list.append(np.array([p for p in prd_bboxs if p[0] >= conf_th]))
        f2_dict = util.calc_f2_score(gt_bboxs_fold_list, prd_bboxs_list, verbose=False)    
        print("fold {}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(fold, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    print()

In [ ]:
pd.set_option('display.max_columns', 40)
f_cols = ["f2"] + ["f2_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
P_cols = ["P"] + ["P_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
R_cols = ["R"] + ["R_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
print("yolov5l, sequence cv method, best CV 0.635 under confidence threshold 0.1")
display(pd.DataFrame.from_dict(perf_dict['l_A']).loc[f_cols,:].style.format("{:.3f}").background_gradient())
display(pd.DataFrame.from_dict(perf_dict['l_A']).loc[P_cols,:].style.format("{:.3f}").background_gradient())
display(pd.DataFrame.from_dict(perf_dict['l_A']).loc[R_cols,:].style.format("{:.3f}").background_gradient())

In [ ]:
gt_sizes = [(g[:,2] * g[:,3]).tolist() for g in gt_bboxs_list if len(g) > 0]
gt_sizes = np.concatenate(gt_sizes).tolist()
pd.Series(gt_sizes).describe()

In [ ]:
# larger box requires larger confidence to stay

In [ ]:
key = "l_A"
conf_list = [] # conf, size
size_list = []
conf_th = 0.05
for index in df.index:
    prd_bboxs = df.loc[index, key]
    conf_list.extend([p[0] for p in prd_bboxs if p[0] >= conf_th])
    size_list.extend([p[3] * p[4] for p in prd_bboxs if p[0] >= conf_th])

In [ ]:
np.min(size_list)

In [ ]:
np.corrcoef(conf_list, size_list)

In [ ]:
size_bucket = [size for i, size in enumerate(size_list) if (conf_list[i] <= 0.2 and conf_list[i] >= 0.1)]
len(size_bucket)

In [ ]:
len([s for s in size_bucket  if s <= 700])

# Check Performance

In [ ]:
# util.save_pickle(df, "df_agg.pkl")

In [ ]:
df['tp'] = 0
df['fp'] = 0
df['fn'] = 0
for key in tqdm(df_d.keys()):
    df[f'{key}_tp'] = df[f'{key}_tps'].apply(np.sum)
    df[f'{key}_fp'] = df[f'{key}_fps'].apply(np.sum)
    df[f'{key}_fn'] = df[f'{key}_fns'].apply(np.sum)
    df['tp'] = df['tp'] + df[f'{key}_tp']
    df['fp'] = df['fp'] + df[f'{key}_fp']
    df['fn'] = df['fn'] + df[f'{key}_fn']

In [ ]:
color_pred = [(0,125,0)]    
color_real = [(255,0,0)]
def plot_img(df, idx, keys, save=False, pre=""):
    row = df.loc[idx]
    path = row["image_path"]
    img = cv2.imread(str(path))[...,::-1]
    real_bboxes = row["real_bbox"]
    preds = []
    for key in keys:
        if len(row[key]) > 0:
            preds = preds + row[key]
    bboxes = [p[1:5] for p in preds]
    confis = [p[0] for p in preds]
    print(f"video {row.video_id}, sequence {row.sequence}, frame {row.video_frame}")
    img_plot = util.show_img(img, 
                          bboxes + real_bboxes, 
                          confis + [1] * len(real_bboxes),
                          color_pred  * len(bboxes) + color_real * len(real_bboxes), 
                          bbox_format='coco')
    img_plot.save(f"./pic_check/{pre}v{row.video_id}_frame{row.video_frame}_seq{row.sequence}.jpg")
    display(img_plot)

In [ ]:
for i, iou in tqdm(enumerate(np.arange(0.3,0.85,0.05))):
    df[f"f2_{iou:.2f}"] = 0
    df[f"P_{iou:.2f}"] = 0
    df[f"R_{iou:.2f}"] = 0
    for idx in df.index:
        tp = 0
        fp = 0
        fn = 0
        for key in df_d.keys():
            tp += df.loc[idx, f'{key}_tps'][i]
            fp += df.loc[idx, f'{key}_fps'][i]
            fn += df.loc[idx, f'{key}_fns'][i]
        df.loc[idx, f"f2_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=2)
        df.loc[idx, f"P_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=0)
        df.loc[idx, f"R_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=100)

In [ ]:
df["f2"] = df[[f'f2_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df["P"] = df[[f'P_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df["R"] = df[[f'R_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df.head()

In [ ]:
show_col = ["video_id","sequence","video_frame","num_bbox", "fold_V", "fold_A", "fold_C", "f2", "P", "R"]

In [ ]:
sub_col = ["fp"] + [key+"_fp" for key in th_d.keys()]
sub_col = sub_col + ["tp"] + [key+"_tp" for key in th_d.keys()]
sub_col = sub_col + ["fn"] + [key+"_fn" for key in th_d.keys()]

## most FP among empy group

In [ ]:
df_neg = df.query("num_bbox == 0").copy()
df_neg.sort_values("fp", ascending=False, inplace=True)
df_neg.reset_index(inplace=True, drop=True)
df_neg[show_col + sub_col].head()

In [ ]:
if not os.path.exists("./pic_check/"):
    os.makedirs("./pic_check/")
    
for i in range(5):
    plot_img(df_neg, i, th_d.keys(), save=True, pre="MostFPinNoBBOXPic_")

## Most FP among non-empty group

In [ ]:
df_pos = df.query("num_bbox > 0").copy()
df_pos.sort_values("fp", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="MostFPinBBOXPic_")

## Most FN among non-empty group

In [ ]:
df_pos = df.query("num_bbox > 0").copy()
df_pos.sort_values("fn", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head(10)

In [ ]:
for i in range(10):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="MostFN_")

## worst F2 with real bbox at least 3

In [ ]:
df_pos = df.query("num_bbox >= 3").copy()
df_pos.sort_values("f2", ascending=True, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="WorstF2inNumBBOXg3_")

## worst F2 with real bbox at least 5

In [ ]:
df_pos = df.query("num_bbox >= 5").copy()
df_pos.sort_values("f2", ascending=True, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
display(df_pos[show_col + sub_col].head())
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="WorstF2inNumBBOXg5_")

## worst F2 with real bbox at least 8

In [ ]:
df_pos = df.query("num_bbox >= 8").copy()
df_pos.sort_values("f2", ascending=True, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
display(df_pos[show_col + sub_col].head())
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="WorstF2inNumBBOXg8_")

## worst F2 with real bbox at least 12

In [ ]:
df_pos = df.query("num_bbox >= 12").copy()
df_pos.sort_values("f2", ascending=True, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
display(df_pos[show_col + sub_col].head())
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="WorstF2inNumBBOXg12_")

## Most TP

In [ ]:
df_pos = df.query("num_bbox>=0").copy()
df_pos.sort_values("tp", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
display(df_pos[show_col + sub_col].head())
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="MostTP_")

## worst F2 with real bbox at least 3 in fold 4

In [ ]:
df_pos = df.query("num_bbox >= 3 and fold_A==4").copy()
df_pos.sort_values("f2", ascending=True, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_pos, i, th_d.keys(), save=True, pre="WorstF2inNumBBOXg3CV4_")

# NoBBOX case with many FP

In [ ]:
df.query("num_bbox==0")['fp'].describe()

In [ ]:
df.query("num_bbox==0")["fp"].value_counts()

In [ ]:
df.query("num_bbox==0 and fp>22").shape, df.query("num_bbox>0").shape

In [ ]:
df_highFPNoBB = df.query("num_bbox==0 and fp>22")[["video_id","video_frame","sequence"]].copy()
df_highFPNoBB['highFBNoBB'] = True

In [ ]:
df_highFPNoBB.apply(lambda x: x.unique())

In [ ]:
df_highFPNoBB.to_csv("../input/df_highFPNoBB.csv", index=False)

# check extreme

In [ ]:
df["tp_ratio"] = df["tp"] / df["tp"].sum()
df["fp_ratio"] = df["fp"] / df["fp"].sum()
df["fn_ratio"] = df["fn"] / df["fn"].sum()
util.f_beta(df["tp"].sum(), df["fp"].sum(), df["fn"].sum(), 2)

In [ ]:
df["size"] = 1
df["has_annotations"] = df["num_bbox"] > 0
seq_ratio = df.groupby(["sequence", "video_id","fold_A
                        "])[["size","has_annotations","num_bbox","tp_ratio", "fp_ratio", "fn_ratio"]].sum()
seq_ratio["ratio"] = seq_ratio["has_annotations"] / seq_ratio["size"]
seq_ratio["f2"] = seq_ratio.apply(lambda row: util.f_beta(row.tp_ratio, row.fp_ratio, row.fn_ratio, beta=2), axis=1) # f2 is invariant
seq_ratio.style.background_gradient()

In [ ]:
df.shape

In [ ]:
util.f_beta(df["tp"].sum(), df["fp"].sum(), df["fn"].sum(), 2)

# pick hold out test sequence

In [ ]:
# < 10% ratio, 0.5 ~ 0.6 cv score
seqs = [996, 15827]
df_seq = df.query("sequence in @seqs").copy()
print(df_seq.shape[0], df_seq.shape[0]/df.shape[0])
gt_bboxs_seq_list = []
for idx in range(len(df_seq)):
    row = df_seq.iloc[idx]
    gt_bboxs = row["real_bbox"]
    gt_bboxs_seq_list.append(np.array(gt_bboxs))

for key in tqdm(df_d.keys()):
    conf_th = th_d[key]
    prd_bboxs_list = []
    for index in df_seq.index:
        prd_bboxs = df_seq.loc[index, key]
        prd_bboxs_list.append(np.array([p for p in prd_bboxs if p[0] >= conf_th]))
    f2_dict = util.calc_f2_score(gt_bboxs_seq_list, prd_bboxs_list, verbose=False)    
    print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(key, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))

# pick validation sequence

In [ ]:
# 18% ratio, ~ 0.6 cv score
seqs = [53708, 59337, 45518,  26651, 15827]
df_seq = df.query("sequence in @seqs").copy()
print(df_seq.shape[0], df_seq.shape[0]/df.shape[0])
gt_bboxs_seq_list = []
for idx in range(len(df_seq)):
    row = df_seq.iloc[idx]
    gt_bboxs = row["real_bbox"]
    gt_bboxs_seq_list.append(np.array(gt_bboxs))

for key in tqdm(df_d.keys()):
    conf_th = th_d[key]
    prd_bboxs_list = []
    for index in df_seq.index:
        prd_bboxs = df_seq.loc[index, key]
        prd_bboxs_list.append(np.array([p for p in prd_bboxs if p[0] >= conf_th]))
    f2_dict = util.calc_f2_score(gt_bboxs_seq_list, prd_bboxs_list, verbose=False)    
    print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(key, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))

In [ ]:
val_seq = [53708, 59337, 45518,  26651, 15827]
df["fold_3"] = 0
df.loc[df.query(f"sequence in {val_seq}").index, "fold_3"] = 1

In [ ]:
df["fold_3"].value_counts(normalize=True)

In [ ]:
idx = pd.IndexSlice
stats = df.groupby("fold_3")[["num_bbox", "has_annotations"]].agg(["count", "sum"])
stats.loc[:,idx["num_bbox","ratio"]] = stats.loc[:,idx["num_bbox","sum"]]/ stats.loc[:,idx["num_bbox","count"]]
stats.loc[:,idx["has_annotations","ratio"]] = stats.loc[:,idx["has_annotations","sum"]]/ stats.loc[:,idx["has_annotations","count"]]
stats.loc[:,idx[["num_bbox","has_annotations"],["count", "sum", "ratio"]]]

In [ ]:
for f in [0, 1]:
    df_fold = df.query("fold_3 == @f").copy()
    gt_bboxs_seq_list = []
    for idx in range(len(df_fold)):
        row = df_fold.iloc[idx]
        gt_bboxs = row["real_bbox"]
        gt_bboxs_seq_list.append(np.array(gt_bboxs))

    key = 'l_A'
    conf_th = th_d[key]
    prd_bboxs_list = []
    for index in df_fold.index:
        prd_bboxs = df_fold.loc[index, key]
        prd_bboxs_list.append(np.array([p for p in prd_bboxs if p[0] >= conf_th]))
    f2_dict = util.calc_f2_score(gt_bboxs_seq_list, prd_bboxs_list, verbose=False)    
    print("fold {}, size {}, model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(f, df_fold.shape[0], key, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))

In [ ]:
seq_fold = df.groupby("sequence")["fold_3"].last().to_dict()
seq_fold

In [ ]:
df.groupby("fold_3")["video_id"].unique()

In [ ]:
util.save_pickle(seq_fold,"../input/fold_test_2.pkl")

In [ ]:
list(zip(range(3), 'abcd'))

# WBF

In [ ]:
def run_wbf(bboxes, confs, image_size=1280, iou_thr=0.4, skip_box_thr=0.7, weights=None, conf_type='avg'):
    if len(bboxes) == 1:
        return bboxes[0], confs[0], []    
    bboxes =  [bbox / (image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(len(conf)) for conf in confs]
    bboxes, scores, labels = weighted_boxes_fusion(bboxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr,conf_type=conf_type)
    bboxes = bboxes * (image_size-1)
    bboxes = bboxes.astype(int)
    if len(bboxes) > 0:
        bboxes[:,2:] = bboxes[:,2:] - bboxes[:,:2]
    wbf_r = [[scores[i]] + bboxes[i].tolist() for i,s in enumerate(scores)]
    return wbf_r

In [ ]:
def prepare(row, keys=th_d.keys()):
    bbox = []
    confs = []
    for key in keys:
        bbox_tmp = np.array([p[1:] for p in row[key]])
        if len(bbox_tmp) > 0:
            bbox_tmp[:,2:] = bbox_tmp[:,2:] + bbox_tmp[:,:2]
        bbox.append(bbox_tmp)
        confs.append(np.array([p[0] for p in row[key]]))
    return bbox, confs

In [ ]:
iou_ths = [0.3,0.5,0.7]
skip_box_ths = [0.3, 0.4, 0.5,0.7]
conf_types = ["max", "avg"]
for iou_th in iou_ths:
    for skip_box_th in skip_box_ths:
        for conf_type in conf_types:
            df_plain['wbf'] = df_plain.apply(lambda row: run_wbf(prepare(row)[0], prepare(row)[1], iou_thr=iou_th, skip_box_thr=skip_box_th, conf_type=conf_type), axis=1 )
            prd_bboxs_list = [np.array(p) for p in df_plain['wbf'].values]
            tps, fps, fns = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
            precision = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=0)
            recall = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=100)    
            score = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=2) 
            print(f"wbf iou_th {iou_th}, skip_box_th {skip_box_th}, conf_type {conf_type}, pred: f2 {score:.4f}, precision {precision:.2f}, recall {recall:.2f} ")

In [ ]:
iou_ths = [0.3,0.5,0.7]
skip_box_ths = [0.4]
conf_types = ["max", "avg"]
for iou_th in iou_ths:
    for skip_box_th in skip_box_ths:
        for conf_type in conf_types:
            df_plain['wbf'] = df_plain.apply(lambda row: run_wbf(prepare(row)[0], prepare(row)[1], iou_thr=iou_th, skip_box_thr=skip_box_th, conf_type=conf_type), axis=1 )
            prd_bboxs_list = [np.array(p) for p in df_plain['wbf'].values]
            tps, fps, fns = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
            precision = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=0)
            recall = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=100)    
            score = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=2) 
            print(f"wbf iou_th {iou_th}, skip_box_th {skip_box_th}, conf_type {conf_type}, pred: f2 {score:.4f}, precision {precision:.2f}, recall {recall:.2f} ")

In [ ]:
iou_th=0.5
skip_box_th=0.4
conf_type='avg'
keys = ["l_A","l_V","s_A"]
df_plain['wbf'] = df_plain.apply(lambda row: run_wbf(prepare(row,keys = keys)[0], prepare(row,keys = keys)[1], iou_thr=iou_th, skip_box_thr=skip_box_th, conf_type=conf_type), axis=1 )
prd_bboxs_list = [np.array(p) for p in df_plain['wbf'].values]
tps, fps, fns = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
precision = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=0)
recall = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=100)    
score = util.f_beta(np.sum(tps), np.sum(fps), np.sum(fns), beta=2) 
print(f"wbf iou_th {iou_th}, skip_box_th {skip_box_th}, conf_type {conf_type}, pred: f2 {score:.4f}, precision {precision:.2f}, recall {recall:.2f} ")